# Setup

TODO: change path `/content/drive/MyDrive/`

In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/
%ls

Mounted at /content/drive
/content/drive/MyDrive
 checkpoints_adapter_compacter/      'Colab Notebooks'/
 checkpoints_adapter_compacter_pp/    COMBINED/
 checkpoints_adapter_houlsby_inv/     custom_adapters/
 checkpoints_adapter_houlsby_inv2/    CustomTrainerCallback.py
 checkpoints_adapter_mam/             model_inference.py
 checkpoints_adapter_pfeiffer_inv/    ParaphrasePipeline.py
 checkpoints_adapter_prefix_tuning/   __pycache__/


In [2]:
!pip install transformers
!pip install adapter-transformers
!pip install datasets
!pip install pytorch_lightning
!pip install sentencepiece

     |████████████████████████████████| 4.0 MB 5.4 MB/s 
     |████████████████████████████████| 596 kB 45.5 MB/s 
     |████████████████████████████████| 895 kB 40.2 MB/s 
     |████████████████████████████████| 77 kB 6.6 MB/s 
     |████████████████████████████████| 6.6 MB 32.2 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 3.9 MB 5.2 MB/s 
     |████████████████████████████████| 325 kB 5.3 MB/s 
     |████████████████████████████████| 1.1 MB 41.6 MB/s 
     |████████████████████████████████| 136 kB 45.6 MB/s 
     |████████████████████████████████| 212 kB 46.4 MB/s 
     |████████████████████████████████| 127 kB 49.0 MB/s 
     |████████████████████████████████| 271 kB 49.5 MB/s 
     |████████████████████████████████| 94 kB 3.4 MB/s 
     |████████████████████████████████| 144 kB 49.5 MB/s 
  Attempting uninstall: urllib3
    Found exis

In [3]:
!nvidia-smi

Fri Apr 29 03:40:58 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P8    26W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
%load_ext autoreload
%autoreload 2

In [5]:
# import transformers
# import datasets

# from transformers import (
#     # AdamW, 
#     T5Model, 
#     T5ForConditionalGeneration, 
#     T5AdapterModel, 
#     T5Tokenizer, 
#     get_linear_schedule_with_warmup,
#     TrainingArguments, 
#     AdapterTrainer,
#     Trainer
# )

# from dataprovider.DataProvider import DatasetProvider

# Load Data

In [22]:
import pandas as pd
import numpy as np
from datasets import Dataset

In [8]:
from transformers import T5Tokenizer

tokenizer = T5Tokenizer.from_pretrained(base_path)

Downloading:   0%|          | 0.00/773k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

In [9]:
df_test = pd.read_csv('COMBINED/test.tsv', sep = '\t', names=['in', 'expected'])
df_train = pd.read_csv('COMBINED/train.tsv', sep = '\t', names=['in', 'expected'])
df_test

,in,expected
0,"PCCW's chief operating officer, Mike Butcher, ...",Current Chief Operating Officer Mike Butcher a...
1,The world's two largest automakers said their ...,Domestic sales at both GM and No. 2 Ford Motor...
2,According to the federal Centers for Disease C...,The Centers for Disease Control and Prevention...
3,A tropical storm rapidly developed in the Gulf...,A tropical storm rapidly developed in the Gulf...
4,The company didn't detail the costs of the rep...,But company officials expect the costs of the ...
...,...,...
5172,Twice Sparrow sold the island twice to Thomas ...,Sparrow twice sold the island to Thomas Polloc...
5173,The name in Tupi means `` insensitive stone ''...,"The name in Tupi means '' hard stone `` , '' i..."
5174,"The company has branches in Tokyo , based in t...",The company has branches in Tokyo based in Sai...
5175,The modern coat of arms of Bavaria was designe...,The modern coat of arms of Bavaria was designe...


# Load Saved Model

In [6]:
base_path = "t5-small"
adapter_path = "paraphrase"

In [10]:
from transformers.adapters import T5AdapterModel

In [11]:
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"

TODO: change the `saved_model_dir` to match the path to your saved model

In [12]:
saved_model_dir = "/content/drive/MyDrive/custom_adapters/houlsby-inv-sm" 

model = T5AdapterModel.from_pretrained(base_path)
model.load_adapter(f"{saved_model_dir}")
model.set_active_adapters(adapter_path)
print(model.active_adapters)
model.to(device)

Downloading:   0%|          | 0.00/231M [00:00<?, ?B/s]

Some weights of the model checkpoint at t5-small were not used when initializing T5AdapterModel: ['decoder.block.0.layer.1.EncDecAttention.relative_attention_bias.weight']
- This IS expected if you are initializing T5AdapterModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing T5AdapterModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of T5AdapterModel were not initialized from the model checkpoint at t5-small and are newly initialized: ['decoder.embed_tokens.weight', 'encoder.embed_tokens.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Stack[paraphrase]


T5AdapterModel(
  (shared_parameters): ModuleDict()
  (transformer): T5Model(
    (shared_parameters): ModuleDict()
    (invertible_adapters): ModuleDict(
      (paraphrase): NICECouplingBlock(
        (F): Sequential(
          (0): Linear(in_features=256, out_features=128, bias=True)
          (1): Activation_Function_Class(
            (f): SiLUActivation()
          )
          (2): Linear(in_features=128, out_features=256, bias=True)
        )
        (G): Sequential(
          (0): Linear(in_features=256, out_features=128, bias=True)
          (1): Activation_Function_Class(
            (f): SiLUActivation()
          )
          (2): Linear(in_features=128, out_features=256, bias=True)
        )
      )
    )
    (shared): Embedding(32128, 512)
    (encoder): T5Stack(
      (invertible_adapters): ModuleDict(
        (paraphrase): NICECouplingBlock(
          (F): Sequential(
            (0): Linear(in_features=256, out_features=128, bias=True)
            (1): Activation_Functio

If you have the `model_inference.py` file, you could import the inference function with:
- `from model_inference import inference`

In [23]:
def generate_predictions(model, tokenizer, row:pd.Series, device="cpu", num_return_sequences=5, verbose=True):
    """
    Given a row of test dataframe, generate predictions
    Args:
        model (transformers.AutoAdapterModel)
        tokenizer (transformers.AutoTokenizer or T5Tokenizer)
        row (pd.Series): a row of a pd.DataFrame
        device (str): device the model is on "cpu" or "cuda" 
        num_return_sequences (int): number of predictions to make
        verbose (bool): whether to print outputs
    Returns:
        list of prediction strings
    """

    if verbose is True:
        print('Input: ', row['in'])
    
    to_model = 'paraphrase: ' + row['in']

    # sentence = 'paraphrase: We should go to the movies today because it is raining.'
    encoding = tokenizer(row['in'], return_tensors="pt")
    
    # Push tensors to device
    for key, value in encoding.items():
        encoding[key] = value.to(device)
    
    # Generate
    input_ids, attention_masks = encoding["input_ids"], encoding["attention_mask"]
    out = model.generate(input_ids=input_ids, do_sample=True, attention_mask=attention_masks, max_length=512,
                        top_k=250, top_p=0.99, early_stopping=True, num_return_sequences=num_return_sequences)
    
    # Decode generated predictions
    predictions = []
    for p in out:
        pred = tokenizer.decode(p, skip_special_tokens=True)
        predictions.append(pred)
        if verbose is True:
            print('Prediction: ', pred)
        
    if verbose is True:
        print('Expected: ', row['expected'], "\n")

    return predictions


def inference(model, tokenizer, df_test, num_examples=1, device="cpu", verbose=True) -> pd.DataFrame:
    """
    Generate and print output for a few of the test dataset 
    Args:
        model (transformers.AutoAdapterModel)
        tokenizer (transformers.AutoTokenizer or T5Tokenizer)
        df_test (pd.DataFrame)
        num_examples (int): number of test sentences to generate paraphrase for
        device (str): device the model is on "cpu" or "cuda"
    Returns:
        df_test_head_copy (pd.DataFrame): a copy of the first num_examples rows of the df_test
            dataframe with the predictions appended as additional columns pred1, pred2, pred3, ...
    """

    num_return_sequences = 5
    
    df_test_head_copy = df_test.head(num_examples).copy(deep=True)

    predictions = df_test_head_copy.apply(lambda row: 
        generate_predictions(model, tokenizer, row, 
            device=device, num_return_sequences=num_return_sequences, 
            verbose=verbose
        ), 
        axis=1
    )

    df_test_head_copy[[f"pred{i}" for i in range(1,num_return_sequences+1)]] = pd.DataFrame(np.vstack(predictions))

    return df_test_head_copy



# Generate Predictions with `inference()`

In [24]:
preds = inference(model, tokenizer, df_test, num_examples=10, device=device, verbose=True)

Input:  PCCW's chief operating officer, Mike Butcher, and Alex Arena, the chief financial officer, will report directly to Mr So.
Prediction:  The CCW Chief Operating Officer and Chairman of PCCW and the chief financial officer, Mike Butcher, chief operating officer of PCCW, and Alex Arena, chief financial officer, will report directly to Mr Socher.
Prediction:  The chief operating officer of PCCW's chief operating officer, Mike Butcher, and Alex Arena, the chief financial officer, will be named Director with a video to Mr Socher and Alex Arena, the chief financial officer of PCCW's PCCW chief operating officer Mike Butcher, and Alex Arena in the op.
Prediction:  The Chief Operating Officer and Chief Financial Officer of PCCW's chief operating officer Mike Butcher, and Alex Arena, the chief financial officer of PCCW, will have the option of reporting directly to Mr Socher.
Prediction:  The chief operating officer for PCCW's chief operating officer, Mike Butcher, and Alex Arena (finance

In [25]:
preds

,in,expected,pred1,pred2,pred3,pred4,pred5
0,"PCCW's chief operating officer, Mike Butcher, ...",Current Chief Operating Officer Mike Butcher a...,The CCW Chief Operating Officer and Chairman o...,The chief operating officer of PCCW's chief op...,The Chief Operating Officer and Chief Financia...,The chief operating officer for PCCW's chief o...,The Chief Operating Officer (Cespionage office...
1,The world's two largest automakers said their ...,Domestic sales at both GM and No. 2 Ford Motor...,The world's two largest automakers said U.S. s...,It's the third largest automaker in the world ...,The world's largest automaker said U.S. sales ...,The world's two largest automakers say by late...,The world's two largest automakers said their ...
2,According to the federal Centers for Disease C...,The Centers for Disease Control and Prevention...,"In 2002, 19 reported cases of measle of measle...",According to the Federal Centers for Disease C...,The federal Centers for Disease Control and Pr...,Selon the federal Centers for Disease Control ...,According to the Federal Centers for Disease C...
3,A tropical storm rapidly developed in the Gulf...,A tropical storm rapidly developed in the Gulf...,The tropical storm had developed Sunday in the...,Sung by Texas or Louisiana by Monday night nig...,It was expected to hit somewhere along the Tex...,It has been reported that a tropical storm wou...,The Tropical storm rapidly developed in the Gu...
4,The company didn't detail the costs of the rep...,But company officials expect the costs of the ...,The company didn't detail details the costs fo...,The company disclosed the cost of the replacem...,The company did not detail the cost of the hom...,The company wasn't detailed the possible costs...,The company had given no details about the cos...
5,The settling companies would also assign their...,"Under the agreement, the settling companies wi...",The settlement companies also would assign the...,"He added that however, their settlement compan...",The settling companies would charge the settle...,The settling companies would also assign their...,The settling companies would also assign its p...
6,Air Commodore Quaife said the Hornets remained...,Air Commodore John Quaife said the security op...,Air Commodore Quaife said the Hornets were on ...,He said the Hornets were on three minute alert...,Commodore Quaife stated that the Hornets kept ...,The Hornets remain on 3-minute alert throughou...,The Hornets are with three seconds off to rece...
7,A Washington County man may have the countys f...,The countys first and only human case of West ...,The health Department said the health departme...,A Washington County man may have the countys f...,The health department said a Washington County...,A Washington County resident may have the coun...,A Washington County man who visited from a Was...
8,Moseley and a senior aide delivered their summ...,General Moseley and a senior aide presented th...,Moseley and an a senior aide and d' 'Missieley...,Moseley & Joseph Moseley and a senior aide end...,Moseley and a senior aide and a Senior aide on...,Moseley and a senior aide by several years of ...,Moseley and one &
9,The broader Standard & Poor's 500 Index <.SPX>...,The technology-laced Nasdaq Composite Index .I...,A 9.97.02..SPX> gave Standard & Poors 500 Inde...,The standard & Poor's 500 Index.SPX> was 0.46 ...,The overall Standard & Poor's 500 Index.SPX> w...,The Greater Standard & Poor's 500 Index.SPX> w...,"The scale.SPX> Index was 0.46 points lower, or..."


# Generate Predictions with a Custom Pipeline

If you have the `ParaphrasePipeline.py` file, you could import this call as:
- `from ParaphrasePipeline import ParaphrasePipeline`

In [15]:
from transformers import Pipeline

class ParaphrasePipeline(Pipeline):
    def _sanitize_parameters(self, **pipeline_parameters):

        print("pipeline parameters: ")
        print(pipeline_parameters)
        return dict(), dict(), dict()

    def preprocess(self, input_, **preprocess_parameters):
        """Tokenize and convert input into torch encodings"""
        print("preprocess:")

        device = self.device

        if isinstance(input_, str):
            encoding = self.tokenizer(f"paraphrase: {input_}", return_tensors="pt")
        
            # Push tensors to device
            for key, value in encoding.items():
                encoding[key] = value.to(device)
            
            input_ids, attention_masks = encoding["input_ids"], encoding["attention_mask"]
            return {"input_ids": input_ids, "attention_masks": attention_masks}
        
        else:
            raise Exception(f"Unhandled input type: {type(input_)}")

    def _forward(self, input_tensors, **forward_parameters):
        """Generate text"""

        print("_forward:")
        print(input_tensors)
        num_return_sequences = 5
        input_ids, attention_masks = input_tensors["input_ids"], input_tensors["attention_masks"]
        out = self.model.generate(input_ids=input_ids, do_sample=True, attention_mask=attention_masks, max_length=512,
                        top_k=250, top_p=0.99, early_stopping=True, num_return_sequences=num_return_sequences)
        return out

    def postprocess(self, model_outputs, **postprocess_parameters):
        """Decode model generated text"""

        print("postprocess")
        print(model_outputs)
        verbose=True
        out = model_outputs
        predictions = []
        for p in out:
            pred = self.tokenizer.decode(p, skip_special_tokens=True)
            predictions.append(pred)
            if verbose is True:
                print('Prediction: ', pred)
        
        return predictions
        # if verbose is True:
        #     print('Expected: ', row['expected'], "\n")

In [16]:
# Initialize pipeline
pipe = ParaphrasePipeline(
    task="text2text-generation",
    model=model,
    tokenizer=tokenizer,
    device=torch.cuda.current_device()
)

pipeline parameters: 
{}


In [17]:
# Generate predictions
pipe("The settling companies would also assign their possible claims against the underwriters to the investor plaintiffs, he added.")

pipeline parameters: 
{}
preprocess:
_forward:
{'input_ids': tensor([[ 3856, 27111,    10,    37,     3, 19691,   688,   133,    92, 12317,
            70,   487,  3213,   581,     8,   365, 12756,     7,    12,     8,
         12024, 16877,     7,     6,     3,    88,   974,     5,     1]],
       device='cuda:0'), 'attention_masks': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1]], device='cuda:0')}
postprocess
tensor([[    0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,    37,     3, 19691,
           688,   133,    92, 12317,    70,   487,  3213,   581,     8,   365,
         12756,     7,    12,  4367, 16877,     7,   113, 12967,    70,  8201,
             3,     5,  1853,     3, 19691,   688,   133,    92, 12317,    70,
          3213,   581,   273,   365, 12756,     7, 13741,    15,  3213,    13,
             8, 16877,     7,    12,     8, 12024, 1

['The settling companies would also assign their possible claims against the underwriters to investors plaintiffs who rejected their contracts. THE settling companies would also assign their claims against those underwriters eventuale claims of the plaintiffs to the investor plaintiffs, he added.',
 'The settle companies would also assign potential claims against the underwriters their settling companies to the investors plaintiffs if the plaintiffs filed claims against the bankers and other investors along the way, he added.',
 "This would increase the amount of water from the settling companies in the companies for the settlement would also assign their possible claims against the underwriters their claims against the underwriters the investor plaintiffs, added a former mastermind. The settling companies would also assign their possible claims against the underwriters to investor plaintiffs, depleting would also have permission of the owners of the settling companies of the ''",
 'Th

In [18]:
# It doesn't always work though:
pipe("Help I've fallen and I can't get up!")

pipeline parameters: 
{}
preprocess:
_forward:
{'input_ids': tensor([[ 3856, 27111,    10,  5090,    27,    31,   162,  9717,    11,    27,
            54,    31,    17,   129,    95,    55,     1]], device='cuda:0'), 'attention_masks': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], device='cuda:0')}
postprocess
tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]])
Prediction:  
Prediction:  
Prediction:  
Prediction:  
Prediction:  


['', '', '', '', '']